In [37]:
from dotenv import load_dotenv
load_dotenv()
import os

In [38]:
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
   
)

In [39]:
import pinecone
from langchain.vectorstores import Pinecone
# find API key in console at app.pinecone.io
YOUR_API_KEY = "0c63694c-7756-401b-a897-03616f43acf9"
# find ENV (cloud region) next to API key in console
YOUR_ENV = "gcp-starter"
index_name = 'udbhata'
embeddings = OpenAIEmbeddings()
pinecone.init(
    api_key=YOUR_API_KEY,
    environment=YOUR_ENV
)
index = pinecone.Index(index_name)

vectorstore = Pinecone(
    index, embeddings.embed_query, text_key="text"
)

c:\Users\sheik jaheer\OneDrive\Desktop\TYNYBAY\udbhata\udbhata-poc-main\myenv\Lib\site-packages\langchain_community\vectorstores\pinecone.py:73: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [46]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
model_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=model_name,temperature=0)
# Build prompt
template = """You are an assistant of a company and you are generating annual  report for current year about profit,loss and revenue ,etc .
Use the following pieces of context to answer the question at the end. If you don't know the answer,
just say that you don't know, don't try to make up an answer. 
 generate the accurate values along with corresponding currency and don't generate description for the question,generate only the values and currency type Example: Profit before tax : 123,874 INR ,if you don't have value just say NA
 and the answer should be from the context given below and it should be exact match.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [47]:
question = """
Generate the annual report of the company for below mentioned points
1. profitNet Sales or Revenue or Revenue from operations(profitNet_sales)
2. PBT or Profit before tax (PBT_Profit)
3. Total assets (Total_assets)
4. Total Liabilities (Total_liabilities)
5. EBITDA or Earnings before tax (EBITDA)
6. Net debt (Net_debt)
7. Total equity (Total_equity)
8. Current Investments (Current_investments)
9. Return on capital employed (ROCE)
10. Debt-equity ratio (Debt_equity_ratio)
11. Does  the company have a formal board diversity policy that clearly requires diversity factors such as gender, race, ethnicity, country of origin, nationality or cultural background in the board nomination process? (Board_diversity_policy), if you have the data mention yes or no
12. Has the company conducted a materiality analysis to identify the most important material issues (economic, environmental, or social) for the company's performance? (Materiality_analysis), if you have the data mention yes or no
13. Does the company publicly disclose details of the materiality analysis, including information on how they conduct the materiality analysis process and progress towards targets or metrics? (Materiality_analysis_details), if you have the data mention yes or no
14. Does the company have Risk Management Committee. (Risk_management_committee), if you have the data mention yes or no
15. Does the company identify emerging risks, and give the mitigation plans for the risks? (Emerging_risks), if you have the data mention yes or no
16. Does the company have a Supplier Code of Conduct and is it publicly available? (Supplier_code_of_conduct), if you have the data mention yes or no
17. Awards for Finance, Risk, Sustainability or Governance(awards)
18. Board experience in Risk Management(Board_experience_in_Risk_Management).value in percentage
19. Female Employees share(female_emp_share).value in percentage
20. Corruption perception index (corruption_perception_index)
"""
resp = qa_chain({"query": question})
print("answer:",resp["result"])
print("source:",)
for i,doc in enumerate(resp["source_documents"]):
    print(i+1,doc.metadata["source"])
    print("")

answer: 1. profitNet Sales or Revenue or Revenue from operations: Rs. 26,524 lakhs
2. PBT or Profit before tax: Rs. 4,654 lakhs
3. Total assets: NA
4. Total Liabilities: NA
5. EBITDA or Earnings before tax: Rs. 5,950 lakhs
6. Net debt: NA
7. Total equity: NA
8. Current Investments: NA
9. Return on capital employed: NA
10. Debt-equity ratio: NA
11. Does the company have a formal board diversity policy that clearly requires diversity factors such as gender, race, ethnicity, country of origin, nationality or cultural background in the board nomination process? No
12. Has the company conducted a materiality analysis to identify the most important material issues (economic, environmental, or social) for the company's performance? Yes
13. Does the company publicly disclose details of the materiality analysis, including information on how they conduct the materiality analysis process and progress towards targets or metrics? No
14. Does the company have Risk Management Committee? Yes
15. Does 

In [45]:
question2="""PBT or Profit before tax (PBT_Profit)"""
question1 = """Does the company have a Supplier Code of Conduct and is it publicly available? (Supplier_code_of_conduct), if you have the data mention yes or no"""
resp = qa_chain({"query": question2})
print("answer:",resp["result"])

answer: PBT or Profit before tax (PBT_Profit) for the current year is $123,874.
